In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

2024-02-23 21:06:46.832536: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
type_counts = application_df['APPLICATION_TYPE'].value_counts()
type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(type_counts[type_counts < 600].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
class_count = application_df['CLASSIFICATION'].value_counts()
class_count.head(20)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
Name: count, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_count_gt_1 = class_count[class_count > 1]
print(class_count_gt_1)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_count[class_count < 300].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
numeric_app_df = pd.get_dummies(application_df)
numeric_app_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,True,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,True,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34295,1,5000,0,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34296,1,5000,0,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34297,1,5000,1,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False


In [10]:
# Split our preprocessed data into our features and target arrays
y = numeric_app_df["IS_SUCCESSFUL"]
X = numeric_app_df.drop(["IS_SUCCESSFUL"],axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

# Second Attempt

In [32]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 6

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 10)                440       
                                                                 
 dense_16 (Dense)            (None, 6)                 66        
                                                                 
 dense_17 (Dense)            (None, 1)                 7         
                                                                 
Total params: 513 (2.00 KB)
Trainable params: 513 (2.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [34]:
# Train the model
fit_model2 = nn2.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 968us/step - loss: 0.5810 - accuracy: 0.7206
Epoch 2/100
804/804 [==============================] - 1s 929us/step - loss: 0.5596 - accuracy: 0.7296
Epoch 3/100
804/804 [==============================] - 1s 940us/step - loss: 0.5558 - accuracy: 0.7305
Epoch 4/100
804/804 [==============================] - 1s 946us/step - loss: 0.5543 - accuracy: 0.7299
Epoch 5/100
804/804 [==============================] - 1s 970us/step - loss: 0.5532 - accuracy: 0.7307
Epoch 6/100
804/804 [==============================] - 1s 936us/step - loss: 0.5521 - accuracy: 0.7317
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5515 - accuracy: 0.7314
Epoch 8/100
804/804 [==============================] - 1s 965us/step - loss: 0.5509 - accuracy: 0.7320
Epoch 9/100
804/804 [==============================] - 1s 955us/step - loss: 0.5503 - accuracy: 0.7327
Epoch 10/100
804/804 [==============================] - 1s 930us/step - los

804/804 [==============================] - 1s 925us/step - loss: 0.5441 - accuracy: 0.7353
Epoch 80/100
804/804 [==============================] - 1s 935us/step - loss: 0.5439 - accuracy: 0.7345
Epoch 81/100
804/804 [==============================] - 1s 937us/step - loss: 0.5441 - accuracy: 0.7341
Epoch 82/100
804/804 [==============================] - 1s 946us/step - loss: 0.5440 - accuracy: 0.7350
Epoch 83/100
804/804 [==============================] - 1s 927us/step - loss: 0.5439 - accuracy: 0.7338
Epoch 84/100
804/804 [==============================] - 1s 975us/step - loss: 0.5435 - accuracy: 0.7353
Epoch 85/100
804/804 [==============================] - 1s 938us/step - loss: 0.5435 - accuracy: 0.7358
Epoch 86/100
804/804 [==============================] - 1s 927us/step - loss: 0.5439 - accuracy: 0.7350
Epoch 87/100
804/804 [==============================] - 1s 940us/step - loss: 0.5434 - accuracy: 0.7353
Epoch 88/100
804/804 [==============================] - 1s 933us/step - loss:

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5489 - accuracy: 0.7283 - 274ms/epoch - 1ms/step
Loss: 0.5489349365234375, Accuracy: 0.7282798886299133


In [17]:
# Export our model to HDF5 file
filepath = "/Users/sophia/Desktop/Deep_Learning_Challenge/h5_models/AlphabetSoupCharity_Optimization.h5"
nn2.save(filepath)

In [35]:

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1=9
hidden_nodes_layer2=18
hidden_nodes_layer3=27
nn = tf.keras.models.Sequential()

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn3.summary()


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 9)                 396       
                                                                 
 dense_19 (Dense)            (None, 18)                180       
                                                                 
 dense_20 (Dense)            (None, 27)                513       
                                                                 
 dense_21 (Dense)            (None, 1)                 28        
                                                                 
Total params: 1117 (4.36 KB)
Trainable params: 1117 (4.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [40]:
# Train the model
fit_model3 = nn3.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5396 - accuracy: 0.7382
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5394 - accuracy: 0.7378
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5393 - accuracy: 0.7387
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5395 - accuracy: 0.7383
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5392 - accuracy: 0.7382
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5392 - accuracy: 0.7383
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5393 - accuracy: 0.7390
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5394 - accuracy: 0.7381
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5392 - accuracy: 0.7383
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5389 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5372 - accuracy: 0.7394
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5373 - accuracy: 0.7396
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5370 - accuracy: 0.7397
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5371 - accuracy: 0.7394
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5372 - accuracy: 0.7382
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5375 - accuracy: 0.7396
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5375 - accuracy: 0.7401
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5373 - accuracy: 0.7397
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5372 - accuracy: 0.7396
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5373 - accuracy: 

In [39]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5509 - accuracy: 0.7279 - 265ms/epoch - 988us/step
Loss: 0.5509141087532043, Accuracy: 0.7279300093650818


In [41]:
# Export our model to HDF5 file
filepath = "/Users/sophia/Desktop/Deep_Learning_Challenge/h5_models/AlphabetSoupCharity_Optimization2.h5"
nn2.save(filepath)